# PhotoEvaporation



GEANT4 安装的数据库中，有一个文件夹 **PhotoEvaporation5.x**，里面文件格式 z[xx].a[xx]，每个文件对应一个核素，z 后面的数值为其质子数，a 后面的数值为核子数。

我们以 *60Ni* 为例，


![picpe1](img/photoevaporationz28a60.png)

该数据库为原子核的给定能级的退激数据，能级信号包含自旋宇称、跃迁角动量等。

- 一个文件的数据可划分为多个子块，每个子块为一个能级。能级中包含基态。
- 每个子块第一行为该能级的信息。
- 子块后面的行为从该能级往下跃迁到较低能级的 gamma 信息。

------


定义同位素能级的数据包含6列：

- 1） 一个整数，定义基态从 0 开始的能级的顺序索引(编号)
- 2） 定义浮动级别（-、+X、+Y、+Z、+U、+V、+W、+R、+S、+T、+A、+B、+C）的字符串
    - "-" 字符串表示它是一个非浮动能级。
- 3） 能级激发能（keV）
- 4） 能级半衰期。-1 半衰期意味着稳定的基态。
- 5） 该能级的自旋宇称。符号表示奇偶性。当 ENSDF 文件中缺少 Jpi 时，将使用值 99。
- 6） n_gammas， 该能级可能的 gamma 退激路径数。
    - 0 意味着该能级在 ENSDF 中没有退激 gamma

------

在定义能级之后，n_gammas 行信数据描述退激 gamma 的信息。gamma 的信息如下:

- 1列: 退激能级的编号
- 2列: 跃迁 gamma 的能量
- 3列: gamma 相对发射强度。
- 4列: 跃迁多极性。1，2，3，4，5，6，7 的多极数表示 E0，E1，M1，E2，M2，E3，M3 单极跃迁，100*Nx+Ny 表示多极跃迁，其中 Ny 和 Ny 取值 1，2、3，4、5、6，7 表示 E0、E1、M1、E2、M2、E3、M3。例如，M1+E2 转换将被写入 304。值为 0 表示未知的多极性。
- 5列: 多极混合比。O 意味着是 E1、M1、E2、M2 跃迁，或者 ENSDF 中没有给出多极混合比。
- 6列: 总内转换系数：α=Ic/Ig
    - 总跃迁是伽马退激和内转换的总和。因此，总分支比与（1+α)*Ig 成比例
    - 给定能级的总分支比总和并不总是达到 100%。它们在内部重新归一。
    - gamma 退激和内转换的相对概率分别为 1/(1+α) 和 α/(1+α)
- 7-16列: 仅当总内转换系数 α 不为 0 时给出
    - K-shell
    - L1-3 shells
    - M1-5 shells
    - Outer shells (shellID = 9 is used, when applicable)
   





## 添加用户数据

默认条件下， GEANT4 会读取文件夹 **PhotoEvaporation5.x** 的数据。在 gamma 谱学研究中，测量的能级信息往往是 ENSDF 中没有的，因此需要读取我们数据表。

**main.cc**

```cpp
  mtrunManager->Initialize();
  // 在内核初始化之后添加数据
  // 在这行代码之后添加

  // userEvaporationData
  //G4NuclearLevelData::GetInstance()->AddPrivateData(Z, A, file_name);
  G4NuclearLevelData::GetInstance()->AddPrivateData(28, 60, "wu_z28.a60");
    

```

文件 **wu_z28.a60** 中，修改了能级 2505 keV 的退激 gamma 分值比。


## 示例

*60Ni* 2505 keV 能级退激在 Ge 探测器上的能谱。


```cpp
void wuPrimaryGeneratorActionAll::GeneratePrimaries(G4Event* anEvent)
{
  G4ParticleTable* particleTable = G4ParticleTable::GetParticleTable();
  G4ParticleDefinition* pp = 0;

  // -------------------------
  
  //自定义带电粒子
  G4int IonZ = 28;
  G4int IonA = 60;
  G4double IonEstar = 2505.753*keV; //exitition energy
  G4double IonQ = 1;
  pp = particleTable->GetIonTable()->GetIon(IonZ, IonA, IonEstar);//4.10.01版本强制 G4IonTable.hh
  particleGun->SetParticleCharge(IonQ);

  if(pp)
    particleGun->SetParticleDefinition(pp);
  else
    G4cout<<"##Null pp in wuPrimaryGeneratorAction::SetParticleGun()"<<G4endl;

  //primary particle kinetic energy
  particleGun->SetParticleEnergy(2.0*MeV);  
 
  // 这里省略代码
  // 。。。。。。
  // 粒子各向同性发射
}
```
    
    
典型数据输出如下:    
![picpe2](img/exampleNi60data.png)    
    
    
比较默认数据库和我们修改数据库， Ge 探测器测量的能谱。其中黑色为 GEANT4 默认 ENSDF 库。
    
![picpe3](img/exampleNi60.png)


**解释为何 gamma 峰存在展宽?**

**图中黑色数据，2505 keV 能级，通过 1173 和 1332 退激到基态，同时也可直接发射 2505 到基态，它们的强度比为 100:2e-6。为何图中 2505 keV 的 gamma 峰这么强?**


In [1]:
!date
!jupyter nbconvert PhotoEvaporation.ipynb --to html

2024年 05月 02日 星期四 10:48:50 CST

[NbConvertApp] Converting notebook PhotoEvaporation.ipynb to html

[NbConvertApp] Writing 567292 bytes to PhotoEvaporation.html

